# A CNN Mnist Model

In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_05b_early_stopping")' FastaiNotebook_05b_early_stopping

## Load data

In [ ]:
import FastaiNotebook_05b_early_stopping
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
// export
import Path
import TensorFlow
import Python

In [ ]:
let plt = Python.import("matplotlib.pyplot")

In [ ]:
let data = mnistDataBunch(flat: false, bs: 512)

In [ ]:
let firstBatch = data.train.first(where: { _ in true })!
let batchShape = firstBatch.xb.shape
let batchSize = batchShape.dimensions[0]
let exampleSideSize = batchShape.dimensions[1]
assert(exampleSideSize == batchShape.dimensions[2])
print("Batch size: \(batchSize)")
print("Example side size: \(exampleSideSize)")

let classCount = firstBatch.yb.shape.dimensions[1]
print("Class count: \(classCount)")

In [ ]:
//export 
public struct CnnModel: Layer {
    public var reshapeToSquare: FAReshape<Float>
    public var conv1: FAConv2D<Float>
    public var conv2: FAConv2D<Float>
    public var conv3: FAConv2D<Float>
    public var conv4: FAConv2D<Float>
    public var pool = FAAvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1)) //TODO: replace by AvgPool
    public var flatten = FAFlatten<Float>()
    public var linear: FADense<Float>
    
    public init(sizeIn: Int, channelIn:Int, channelOut:Int, nFilters:[Int]) {
        reshapeToSquare = FAReshape<Float>([-1, Int32(sizeIn), Int32(sizeIn), Int32(channelIn)])
        conv1 = FAConv2D<Float>(
            filterShape: (5, 5, 1, nFilters[0]), 
            strides: (2, 2), 
            padding: .same, 
            activation: relu)
        conv2 = FAConv2D<Float>(
            filterShape: (3, 3, nFilters[0], nFilters[1]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        conv3 = FAConv2D<Float>(
            filterShape: (3, 3, nFilters[1], nFilters[2]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        conv4 = FAConv2D<Float>(
            filterShape: (3, 3, nFilters[2], nFilters[3]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        linear = FADense<Float>(inputSize: nFilters[3], outputSize: channelOut)
    }
    
    @differentiable
    public func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

In [ ]:
let model = CnnModel(sizeIn:28, channelIn: 1, channelOut: 10, nFilters: [8, 16, 32, 32])

In [ ]:
// Test that data goes through the model as expected.
let predictions = model.applied(to: firstBatch.xb, in: Context(learningPhase: .training))
print(predictions.shape)
print(predictions[0])

# Compare training on CPU and GPU

In [ ]:
let opt = SGD<CnnModel, Float>(learningRate: 0.4)
func modelInit() -> CnnModel { return CnnModel(sizeIn:28, channelIn: 1, channelOut: 10, nFilters: [8, 16, 32, 32]) }

// TODO: When TF-421 is fixed, switch back to the normal `softmaxCrossEntropy`.

@differentiable(vjp: _vjpSoftmaxCrossEntropy)
func softmaxCrossEntropy1<Scalar: TensorFlowFloatingPoint>(
    _ features: Tensor<Scalar>, _ labels: Tensor<Scalar>
) -> Tensor<Scalar> {
    return Raw.softmaxCrossEntropyWithLogits(features: features, labels: labels).loss.mean()
}

@usableFromInline
func _vjpSoftmaxCrossEntropy<Scalar: TensorFlowFloatingPoint>(
    features: Tensor<Scalar>, labels: Tensor<Scalar>
) -> (Tensor<Scalar>, (Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>)) {
    let (loss, grad) = Raw.softmaxCrossEntropyWithLogits(features: features, labels: labels)
    let batchSize = Tensor<Scalar>(features.shapeTensor[0])
    return (loss.mean(), { v in ((v / batchSize) * grad, Tensor<Scalar>(0)) })
}

let learner = Learner(data: data, lossFunction: softmaxCrossEntropy1, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

In [ ]:
// This happens on the GPU (if you have one and it's configured correctly).
// I tried this on a GCE 8vCPU 30GB + Tesla P100:
// - time: ~4.3s
// - nvidia-smi shows ~10% GPU-Util while this is running
time {
    try! learner.fit(1)
}

In [ ]:
// This happens on the CPU.
// I tried this on a GCE 8vCPU 30GB + Tesla P100:
// - time: ~6.3s
// - nvidia-smi shows 0% GPU-Util while this is running
time {
    withDevice(.cpu) {
        try! learner.fit(1)
    }
}

# Collect Layer Activation Statistics

In [ ]:
class ActivationStatistics: LayerDelegate<Tensor<Float>> {
    var activationMeans: [Float] = []
    var activationStds: [Float] = []    
    override func didProduceActivation(_ activation: Tensor<Float>, in context: Context) {
        guard context.learningPhase == .training else { return }
        activationMeans.append(activation.mean().scalar!)
        activationStds.append(activation.standardDeviation().reshaped(to: []).scalar!)
    }
}

In [ ]:
// Utility function for getting all the delegates of a certain type of layer.
// Alternatively, we could ask for all the delegates in the model, but then we'd also get delegates for
// uninteresting layers like reshape layers.
// TODO: I have no idea if it preserves order.
extension KeyPathIterable {
    func layerDelegates<T: FALayer>(of layer: T.Type) -> [WritableKeyPath<Self, LayerDelegate<T.Output>>] {
        return recursivelyAllWritableKeyPaths(to: layer).map { kp in
            return kp.appending(path: \T.delegate)
        }
    }
}

In [ ]:
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy1, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

let interestingLayerDelegates = learner.model.layerDelegates(of: FAConv2D<Float>.self) + [
    \CnnModel.pool.delegate,
    \CnnModel.linear.delegate
]

interestingLayerDelegates.forEach { learner.model[keyPath: $0] = ActivationStatistics() }

In [ ]:
// This LayerDelegate stuff slows it down to ~6s/epoch.
time {
    try! learner.fit(2)
}

In [ ]:
for kp in interestingLayerDelegates {
    plt.plot((learner.model[keyPath: kp] as! ActivationStatistics).activationMeans)
}
plt.legend(Array(1...interestingLayerDelegates.count))
plt.show()

In [ ]:
for kp in interestingLayerDelegates {
    plt.plot((learner.model[keyPath: kp] as! ActivationStatistics).activationStds)
}
plt.legend(Array(1...interestingLayerDelegates.count))
plt.show()

## Export

In [ ]:
notebookToScript(fname: (Path.cwd / "06_cuda.ipynb").string)